In [18]:
import boto3
import json
import os
session = boto3.Session(profile_name=os.environ['AWS_PROFILE'])
s3 = session.client('s3')

bucket_name = 'interview-transcripts-bucket'
key = 'interviewtranscripts/647119219494_20250530-144827.json'

response = s3.get_object(Bucket=bucket_name, Key=key)
transcript_data = json.loads(response['Body'].read().decode('utf-8'))


In [19]:
from pydantic import BaseModel, Field
from typing import List, Dict
class QuestionAssessment(BaseModel):
    question: str = Field(..., description="The question asked during the interview.")
    candidate_response: str = Field(..., description="The response provided by the candidate.")
    answer_correctness: str = Field(..., description="Yes/No indicating whether the response was correct.")
    feedback: str = Field(..., description="Feedback on the response.")

class TopicEvaluation(BaseModel):
    topic: str = Field(..., description="The topic being assessed.")
    assessments: List[QuestionAssessment] = Field(..., description="List of question assessments for this topic.")

class InterviewEvaluation(BaseModel):
    topics: List[TopicEvaluation] = Field(..., description="List of topics with assessments.")

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import json
from google import genai
from dotenv import load_dotenv
import os
import json
import pandas as pd
api_key = os.getenv('GOOGLE_API_KEY')
def evaluate_transcript(transcript):
        evaluation_prompt = f"""
            # Technical Interview Evaluation Task
            ## Interview Transcript
            {transcript}
            ## Evaluation Instructions:
            Analyze this technical interview transcript and provide a structured evaluation of the candidate's performance.
            1. Carefully review the candidate's responses based on the questions in the transcript.
            2. For each question, determine if it was answered correctly (Yes/No).
            3. Assess:
            - Technical accuracy and understanding.
            4. Generate a structured report categorized by topics
            """

        client = genai.Client(api_key=api_key)
        model_response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=evaluation_prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': InterviewEvaluation,
        }
        )
        return model_response

In [21]:
response = evaluate_transcript(transcript_data)

In [22]:
evaluation = json.loads(response.text)
data = []
for topic in evaluation['topics']:
    for assessment in topic['assessments']:
        data.append({
            "Topic": topic['topic'],
            "Question": assessment['question'],
            "Candidate Response": assessment['candidate_response'],
            "Answer Correctness": assessment['answer_correctness'],
            "Feedback": assessment['feedback']
        })
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(data)
output_file = 'result'
output_file_name = f'{output_file}.csv'
df.to_csv(output_file_name, index=False)